In [1]:
import numpy as np
import csv
import nltk
import itertools
from datetime import datetime
import rnn_with_numpy as rnn
reload(rnn)

<module 'rnn_with_numpy' from 'rnn_with_numpy.pyc'>

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/anderson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
SENTENCE_START = 'SETENCE_START'
SENTENCE_END = 'SENTENCE_END'
UNKNOWN_TOKEN = 'UNKNOWN_TOKEN'
VOCAB_SIZE = 10000

In [ ]:
with open('/home/anderson/Dropbox/reddit_comments_filtered.csv', 'rt') as f:
    reader = csv.reader(f, skipinitialspace=True)
    print 'start reading comments! (wait for minutes...)'
    
    # Split full comments into sentences
    sentences = itertools.chain(
        *[nltk.sent_tokenize(x[0].decode('UTF-8').lower()) for x in reader])
    print 'Split Done'
    
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (SENTENCE_START, x, SENTENCE_END)
                    for x in sentences ]
    print 'START & END Done'

start reading comments! (wait for minutes...)
Split Done
START & END Done


In [ ]:
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(st) for st in sentences ]

In [ ]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_freq.items())

In [ ]:
vocab = word_freq.most_common(VOCAB_SIZE-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(UNKNOWN_TOKEN)
word_to_index = dict([(w, i) for i, w in enumerate(index_to_word)])

print "Using vocabulary size %d." % VOCAB_SIZE
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

In [ ]:
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else UNKNOWN_TOKEN for w in sent]

In [ ]:
print "'%s'" % sentences[28]
print "'%s'" % tokenized_sentences[28]

In [ ]:
# Create the training data
x_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

print x_train[0]
print y_train[0]

In [ ]:
reload(rnn)
model = rnn.RNNNumpy(VOCAB_SIZE)
o, s = model.forward_propagation(x_train[9])
p = model.predict(x_train[9])

In [ ]:
model.cross_entropy(x_train[26:27], y_train[26:27])

In [ ]:
model.bptt(x_train[26], y_train[26])